In [1]:
import numpy as np

In [2]:
def hybrid_crossover_frequency(bands_per_decade, hybrid_resolution, max_frequency, min_frequency=0):
    log_band_scaling = 10 ** (0.5 / bands_per_decade)
    # The frequency at which the logspaced bands cover at least one linspaced band
    minimum_bandwidth_frequency = hybrid_resolution / (log_band_scaling - 1 / log_band_scaling)
    first_log_idx = int(np.ceil(bands_per_decade * np.log10(minimum_bandwidth_frequency)))
    last_linear_idx = int(np.floor(minimum_bandwidth_frequency / hybrid_resolution))

    # Since the logspaced bands have pre-determined centers, we can't just start them after the linspaced bands.
    # Often, the bands will overlap at the minimum bandwidth frequency, so we look for the first band
    # that does not overlap, i.e., the upper edge of the last linspaced band is below the lower edge of the first
    # logspaced band
    while (last_linear_idx + 0.5) * hybrid_resolution > 10 ** (
        (first_log_idx - 0.5) / bands_per_decade
    ):
        # Condition is "upper edge of last linear band is higher than lower edge of first logarithmic band"
        last_linear_idx += 1
        first_log_idx += 1

    print(f"Crossover from linear to log at {hybrid_resolution * last_linear_idx}")

    if last_linear_idx * hybrid_resolution > max_frequency:
        last_linear_idx = int(np.floor(max_frequency / hybrid_resolution))
    first_linear_idx = int(np.ceil(min_frequency / hybrid_resolution))
    last_log_idx = round(bands_per_decade * np.log10(max_frequency))
    lin_centers = np.arange(first_linear_idx, last_linear_idx) * hybrid_resolution
    log_centers = 10 ** (np.arange(first_log_idx, last_log_idx + 1) / bands_per_decade)

    centers = np.concatenate([lin_centers, log_centers])

    if centers[0] < min_frequency:
        mask = centers >= min_frequency
        # lowers = lowers[mask]
        centers = centers[mask]
        # uppers = uppers[mask]
    if centers[-1] > max_frequency:
        mask = centers <= max_frequency
        # lowers = lowers[mask]
        centers = centers[mask]
        # uppers = uppers[mask]
    print(f"Using a total of {centers.size} frequencies")

In [3]:
hybrid_crossover_frequency(1000, 1, 40000)

Crossover from linear to log at 455
Using a total of 2399 frequencies
